In [1]:
import numpy as np
from linear import LinintGrow, LinintEqui
from DiscretizeTools import grow_grid, rouwenhorst
from scipy.optimize import root



In [2]:
T = 40
J = 12
JR = 10
NP = 2
NS = 5
NA = 100

γ = 0.5
egam = 1 - 1/γ
ν = 0.335
β = 0.998**5

σθ = 0.23
σϵ = 0.05
ρ = 0.98

α = 0.36
δ = 1 - (1 - 0.0823)**5
Ω = 1.60

al = 0
au = 35
agrow = 0.05

n_p = (1 + 0.01)**5 -1
itermax = 50



In [3]:
NA = 100
NP = 2
NS = 5


r = np.zeros(T+1)
rn = np.zeros(T+1)
w = np.zeros(T+1)
wn = np.zeros(T+1)
pp = np.zeros(T+1)

In [4]:
pop = np.zeros((J, T+1))
m = np.zeros((J, T+1))
aplus = np.zeros((J, NA + 1, NP, NS, T + 1))
c = np.zeros((J, NA + 1, NP, NS, T + 1))
l = np.zeros((J, NA + 1, NP, NS, T + 1))
VV = np.zeros((J, NA + 1, NP, NS, T + 1))
v = np.zeros((J, NA + 1, NP, NS, T + 1))
RHS = np.zeros((J, NA + 1, NP, NS, T + 1))
EV = np.zeros((J, NA + 1, NP, NS, T + 1))

eff = np.zeros(J)

distθ = [0.5,0.5]

θ = np.exp(np.array([-σθ**0.5,σθ**0.5]))

η, π = rouwenhorst(ρ, σϵ, NS)
η = np.exp(η)


for j in range(J):
    pop[j, 0] = 1/(1+n_p)**j
    
for j in range(J):
    m[j, 0] = pop[j,0]/pop[0,0]

a = grow_grid(al, au, agrow, n=NA+1)


for j in range(J):
    for p in range(NP):
        for s in range(NS):
            aplus[j, :, p, s, 0] = np.maximum(a/2, np.ones(NA+1)*a[0]/2)

eff[0] = 1.0000
eff[1] = 1.3527
eff[2] = 1.6952
eff[3] = 1.8279
eff[4] = 1.9606
eff[5] = 1.9692
eff[6] = 1.9692
eff[7] = 1.9392
eff[8] = 1.9007
eff[JR-1:] = 0 

tax = np.ones(T+1)*2
τc = np.ones(T+1)*0.075
τw = np.ones(T+1)*0
τr = np.ones(T+1)*0
τp = np.ones(T+1)*0.1
κ = np.ones(T+1)*0.5

gy = 0.19
by = 0.6/5

K = np.ones(T+1) 
L = np.ones(T+1)
Y = np.ones(T+1)
I = np.ones(T+1)*(n_p+δ)*K

G = np.ones(T+1)*gy*Y[0]
B = np.ones(T+1)*by*Y[0]

pen = np.zeros((J,T+1))
pen[JR-1:] = κ[0]
    


In [5]:
def prices(t):    
    r[t] = Ω*α*(K[t]/L[t])**(α-1)-δ
    w[t] = Ω*(1-α)*(K[t]/L[t])**α
    rn[t] = r[t]*(1-τr[t])
    wn[t] = w[t]*(1-τw[t]-τp[t])
    pp[t] = 1+τc[t]
    return None
    



In [6]:
# current time is t, calculate at which year the agent with age j turns to be age p
def year(t,j,p):
    year = t + p -j
    
    if t == 0 or year <= 0 : year = 0
        
    if t == T or year >= T : year =  T
    
    return year

In [7]:
def valuefunc(ap, cons, lab, j,p,s,t):
    chelp = max(cons,1e-10)
    lhelp = min(max(lab,0),1-1e-10)
    
    tp = year(t,j,j+1)
    
    ial, iar, ϕ = LinintGrow(ap,al,au,agrow,NA)
    
    val = 0
    
    if j < J-1:
        val = max(ϕ*EV[j+1, ial, p, s, tp] + (1-ϕ)*EV[j+1, iar, p, s, tp], 1e-10)**(1-1/γ)/(1-1/γ)

    val = (chelp**ν*(1-lhelp)**(1-ν))**(1-1/γ)/(1-1/γ) + β*val
    return val
    
    
    
    

In [16]:
def margu(cons,lab,it):
    return ν*(cons**ν*(1-lab)**(1-ν))**(1-1/γ)/(pp[it]*cons)


def interpolate(j,t):
    for i in range(NA+1):
        for p in range(NP):
            for s in range(NS):
                RHS[j,i,p,s,t] = 0
                EV[j,i,p,s,t] = 0
                
                for sp in range(NS):
                    chelp = max(c[j,i,p,sp,t],1e-10)
                    lhelp = max(l[j,i,p,sp,t],1e-10)
                    RHS[j,i,p,s,t] += π[s,sp]*margu(chelp,lhelp,t)
                    EV[j,i,p,s,t]  += π[s,sp]*VV[j,i,p,sp,t]
                RHS[j,i,p,s,t] = ((1+rn[t])*β*RHS[j,i,p,s,t])**(-γ)
                EV[j,i,p,s,t] = ((1-1/γ)*EV[j,i,p,s,t])**(1/(1-1/γ))
    
    

In [ ]:
def foc(xin,i, j, p, s, t):
    ap = xin
    tp = year(t,j,j+1)
    vind = v(j, i, p, s, t)
    
    wage = wn[t]*eff[j]*θ[p]*η[s]
    
    available = (1 + rn[t])*a[ia] + pen[j,t] + vind
    
    if j < JR-1:
        lab_com = min(max(ν + (1-ν)*(ap-available)/wage, 0),1-1e-10)
    else:
        lab_com = 0
        
    conscom = max((available + wage*lab_com -ap)/p[t],1e-10) 
    
    ial,iar,φ = LinintGrow(ap, al, au, agrow, NA)
    
    tomorrow = max(φ*RHS[j+1,ial,p,s,tp]+(1-φ)*RHS[j+1, iar,p,s,tp],0)
    
    return margu(cons_com, lab_com, t)**(-γ) - tomorrow
        
        

In [17]:
def solve_household(j, t):
    it = year(t, j, J-1)
    
    for i in range(NA+1):
        aplus[J-1,i,:,:,it] = 0
        c[J-1,i,:,:,it] = ((1+rn[it])*a[i]+ pen[J-1,it] + v[J-1,i,:,:,it])/pp[it]
        l[J-1,i,:,:,it] = 0
        VV[J-1,i,:,:,it] = valuefunc(0, c[J-1, i, 0, 0, t], l[J-1, i, 0, 0, t], J-1, 0,0, it)
    
    interpolate(J-1, it)
    
    for ij in range(J-2, j-1, -1):
        it = year(t, j, ij)
        
        if ij >= JR-1:
            ipmax = 1
            ismax = 1
        else:
            ipmax = NP
            ismax = NS
            
        for i in range(NA+1):
            if (ij >= JR-1 and i == 0 and κ[t] <= 1e-10):
                aplus[ij,i,:,:,it] = 0
                c[ij,i,:,:,it] = 0
                l[ij,i,:,:,it] = 0
                VV[ij,i,:,:,it] = valuefunc(0,0,0,ij,0,0,it)
                continue
            
            for p in range(ipmax):
                for s in range(ismax):
                    res = root(foc, x0 = aplus[j,i,p,s], args = (j,i,p,s), tol = 1e-8)
    
    
    
    

In [18]:
solve_household(8,4)

7


C:\Users\u6621852\AppData\Local\Temp\ipykernel_21556\1762821258.py:7: RuntimeWarning: divide by zero encountered in true_divide
  c[J-1,i,:,:,it] = ((1+rn[it])*a[i]+ pen[J-1,it] + v[J-1,i,:,:,it])/pp[it]
C:\Users\u6621852\AppData\Local\Temp\ipykernel_21556\1594614945.py:2: RuntimeWarning: invalid value encountered in double_scalars
  return ν*(cons**ν*(1-lab)**(1-ν))**(1-1/γ)/(pp[it]*cons)
